# IMPORTANTE: Esto solo es de guía, no aplica para el estudio

In [1]:
import pandas as pd

def read_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as a:
        print("Error reading the CSV file: {a}")
        return None

In [2]:
terminos_df = read_csv_to_dataframe("Terminos_lagoon_TA_DIC_2023_RawData.csv")

In [3]:
print(terminos_df.head())

   sample      date     estuary   area station layer_depth season  \
0  CDL01S  5/3/2020  Candelaria  River   CDL01     Surface    Dry   
1  CDL01F  5/3/2020  Candelaria  River   CDL01      Bottom    Dry   
2  CDL02S  5/3/2020  Candelaria  River   CDL02     Surface    Dry   
3  CDL02F  5/3/2020  Candelaria  River   CDL02      Bottom    Dry   
4  CDL03S  5/3/2020  Candelaria  River   CDL03     Surface    Dry   

   chlorophy_microg_l  cond_microsiemens_cm  depth_m  ...  do_mg_l  sal_psu  \
0                0.36                7015.4    0.464  ...     7.12     3.56   
1                4.19               29886.1    7.792  ...     4.90    16.97   
2                0.92               16691.1    0.453  ...     6.99     8.94   
3                2.23               24847.4    1.261  ...     6.52    13.87   
4                0.58               46341.6    0.465  ...     6.24    28.06   

   sp_cond_microsiemens_cm  turbidity_fnu  temp_c  latitude  longitude  \
0                   6547.7          

In [4]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def two_way_anova_tukey(df, response, factor1, factor2, alpha=0.05):

    # Fit the two-way ANOVA model and generate the ANOVA table
    model = ols(f'{response} ~ C({factor1}) + C({factor2}) + C({factor1}):C({factor2})', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print("\nANOVA Results:\n", anova_table)

    # Run Tukey HSD post-hoc test if significant effects are found
    if (anova_table['PR(>F)'] < alpha).any():
        print("\nSignificant effects detected. Running Tukey HSD post-hoc test:")
        df['Group'] = df[factor1].astype(str) + " - " + df[factor2].astype(str)
        tukey = pairwise_tukeyhsd(df[response], df['Group'], alpha=alpha)
        print(tukey.summary())
    else:
        print("\nNo significant effects detected. Post-hoc test is not required.")

In [5]:
two_way_anova_tukey(terminos_df, response="ta_micromol_kg", factor1="season", factor2="estuary")


ANOVA Results:
                             sum_sq     df          F        PR(>F)
C(season)             7.234655e+06    1.0  45.502782  9.393830e-10
C(estuary)            9.238908e+03    1.0   0.058109  8.099945e-01
C(season):C(estuary)  8.689347e+03    1.0   0.054652  8.156259e-01
Residual              1.621735e+07  102.0        NaN           NaN

Significant effects detected. Running Tukey HSD post-hoc test:
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
      group1             group2        meandiff p-adj    lower     upper   reject
---------------------------------------------------------------------------------
  Dry - Candelaria     Dry - Palizada   31.1111 0.9874 -214.3649  276.5871  False
  Dry - Candelaria Rainy - Candelaria -539.7153 0.0001  -852.637 -226.7935   True
  Dry - Candelaria   Rainy - Palizada -547.4444    0.0   -848.09 -246.7989   True
    Dry - Palizada Rainy - Candelaria -570.8264    0.0 -883.7482 -257.9046   True
    Dry - 